In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import aurora_data_api
from dateutil import parser

In [2]:
nameMap = {
    'MEGAN': "Megan Schacht-bonneau",
    'Megan': "Megan Schacht-bonneau",
    "JORDYS": "Jordys Placido",
    "Jordys": "Jordys Placido",
    "NIKKY": "Nikky Voeuth",
    "Nikky": "Nikky Voeuth"
}

def getDate(inp):
    dateVal = re.findall('[0-9][0-9]?/..?/2?0?21', inp)[0]
    formatted = parser.parse(dateVal).strftime('%Y-%m-%d')
    return formatted

# Assume that 
def getFailureType(inp):
    if inp[0] == 'W':
        return inp[:6]
    else:
        return 'none'
    
def getBatchNumber(inp):
    mostRecentBatch = inp.iloc[0]
    for i in range(len(inp)):
        val = inp.iloc[i]
        if type(val) == str:
            mostRecentBatch = val.split(' ')[1]
        inp.iloc[i] = mostRecentBatch
    return inp

def getTester(inp):
    return nameMap.get(inp, inp)

# Execute a SQL query: helper method to be used by insertEntry
def execute(q):
    try:
        db = aurora_data_api.connect(aurora_cluster_arn = r_ARN, secret_arn=s_ARN, database=dbName)
        c = db.cursor()
        c.execute(q) # Execute the query
        db.commit()
        return c.fetchall() # Display the output of said query
    except Exception as e: # Error occurred during execute
        print('Error with execute:', e)
    
        return (float('inf'), e)
    
def insertEntry(sn, bn, date, tst, ft, fail, msg, dt):
    try:
        q = ''' Insert into QAResults values ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')''' % (sn, bn, date, tst, ft, fail, msg, dt)
        return execute(q)
    except Exception as e:
        print('serialNum', sn)
        print('Issue with insert entry:', e)
    
# Read the path in--get a table formatted to more easily insert entries
def readPath(path, deviceType):
    df = pd.read_excel(path)
#     display(df.head())
    df = df.dropna(how = 'all')
    df = df.drop(columns = ['Unnamed: 5'])
#     df = df[df['Unnamed: 2'] != 'RESULT']
    df.columns = ['BatchNumber', 'SerialNumber', 'FunctionalTest', 'LogMessage', 'Tester']
    df['BatchNumber'] = getBatchNumber(df['BatchNumber'])
    df = df[df['FunctionalTest'] != 'RESULT']
    df['Tester'] = df['Tester'].apply(getTester)
    df['Date'] = df['LogMessage'].apply(getDate)
    df['FailureType'] = df['LogMessage'].apply(getFailureType)
    df['DeviceType'] = deviceType
    
    return df

res1 = readPath('Catalina QC Test 05.03 05.08 2021.xlsx', 'Catalina')
res2 = readPath('Catalina QC Test 05.09 05.15 2021.xlsx', 'Catalina')
res3 = None # NOTE: this table doesn't contain any relevant data
res4 = readPath('Catalina QC Test 05.23 05.29 2021.xlsx', 'Catalina')
res5 = readPath('Catalina QC Test 05.30 06.05 2021.xlsx', 'Catalina')


In [3]:
port = 3306
user = 'admin'
password = 'password'

host = 'dashboarddb.cluster-cyvmmsfjcmrl.us-east-1.rds.amazonaws.com'
dbName = 'db'
r_ARN = 'arn:aws:rds:us-east-1:319481862133:cluster:dashboarddb'
s_ARN = 'arn:aws:secretsmanager:us-east-1:319481862133:secret:dashboardDBKey-6XBatB'
def insertTable(res):
    for i in range(len(res)):
        row = res.iloc[i]
        bn = row['BatchNumber']
        sn = row['SerialNumber']
        tst = row['Tester']
        msg = row['LogMessage']
        ft = row['FunctionalTest']
        fail = row['FailureType']
        date = row['Date']
        dt = row['DeviceType']
        insertEntry(sn, bn, date, tst, ft, fail, msg, dt)
    print('success!')
insertTable(res4)

Error with execute: (<MySQLErrorCodes.ER_WRONG_VALUE_COUNT_ON_ROW: 1136>, "Column count doesn't match value count at row 1")
Error with execute: (<MySQLErrorCodes.ER_WRONG_VALUE_COUNT_ON_ROW: 1136>, "Column count doesn't match value count at row 1")
Error with execute: (<MySQLErrorCodes.ER_WRONG_VALUE_COUNT_ON_ROW: 1136>, "Column count doesn't match value count at row 1")
Error with execute: (<MySQLErrorCodes.ER_WRONG_VALUE_COUNT_ON_ROW: 1136>, "Column count doesn't match value count at row 1")
Error with execute: (<MySQLErrorCodes.ER_WRONG_VALUE_COUNT_ON_ROW: 1136>, "Column count doesn't match value count at row 1")
Error with execute: (<MySQLErrorCodes.ER_WRONG_VALUE_COUNT_ON_ROW: 1136>, "Column count doesn't match value count at row 1")
Error with execute: (<MySQLErrorCodes.ER_WRONG_VALUE_COUNT_ON_ROW: 1136>, "Column count doesn't match value count at row 1")
Error with execute: (<MySQLErrorCodes.ER_WRONG_VALUE_COUNT_ON_ROW: 1136>, "Column count doesn't match value count at row 1")


KeyboardInterrupt: 

In [ ]:
'''
DUPLICATE ENTRIES

Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8T1-GH00-0452-PEMW' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8T1-GH00-0452-MLRG' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8T1-GH00-0452-P52X' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8T1-GH00-0452-A4PA' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8T1-GH00-0402-V7S7' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8T1-GH00-0402-N4GN' for key 'PRIMARY'")
Error with execute: (<MySQLErrorCodes.ER_DUP_ENTRY: 1062>, "Duplicate entry 'G8T1-GH00-0402-N4NE' for key 'PRIMARY'")

'''

In [ ]:
# Mini July Entries
